<center>
<hr>
<h1>Complessità nei sistemi sociali</h1>
<h2>Laurea Magistrale in Fisica Dei Sistemi Complessi</h2>
<h2>A.A. 2018/19</h2>
<h3>Daniela Paolotti & Michele Tizzoni</h3>
<h3>Notebook 5 - Basic network analysis with NetworkX, assortativity, resilience to attacks</h3>
<hr>
</center>

In [ ]:
import networkx as nx
import seaborn as sns

In [ ]:
%pylab inline

## Connectivity and clustering of a graph

We study the network of coauthorships of Astro-Ph, from the SNAP database.

In [ ]:
filepath='./../network_data/ca-AstroPh.txt'

In [ ]:
G=nx.Graph()

In [ ]:
fh=open(filepath,'r')
for line in fh.readlines():
    s=line.strip().split()
    if s[0]!='#':
        origin=int(s[0])
        dest=int(s[1])
        G.add_edge(origin,dest)
fh.close()

In [ ]:
print("The graph has", len(G), "nodes and",len(G.edges()),"edges")

In [ ]:
print("Is the graph simply connected?", nx.is_connected(G))

### Show the components of the graph

In [ ]:
print("The graph has", nx.number_connected_components(G),"connected components")

In [ ]:
for k in nx.connected_components(G):
    print(len(k))

### Extract the largest Connected Component as a subgraph

In [ ]:
nx.connected_component_subgraphs(G)

In [ ]:
graphs = list(nx.connected_component_subgraphs(G))

In [ ]:
graphs

In [ ]:
H=graphs[0]

In [ ]:
len(H)

In [ ]:
print(len(G)-len(H))

In [ ]:
print("Check that the graph is now connected")
nx.number_connected_components(H)

## Global clustering coefficient

The global clustering coefficient measures the number of triangles in the network and it's defined as: 
<center>
$C_\Delta = \frac{3 \times \text{triangles}}{\text{triplets}}$
</center> 

In [ ]:
nx.triangles(H)

How many triangles are there in the whole network?

In [ ]:
tt=sum(list(nx.triangles(H).values()))

In [ ]:
tt/3

The transitivity is the fraction of all possible triangles in the network.

In [ ]:
nx.transitivity(H)

## Average clustering coefficient
As an alternative to the global clustering coefficient, the overall level of clustering in a network is measured by Watts and Strogatz as the average of the local clustering coefficients of all the vertices $n$:

<center>
$\bar{C} = \frac{1}{n}\sum_{i=1}^{n} C_i.$
</center>

It is worth noting that this metric places more weight on the low degree nodes, while the transitivity ratio places more weight on the high degree nodes. In fact, a weighted average where each local clustering score is weighted by $k_i(k_i-1)$ is identical to the global clustering coefficient.

In [ ]:
print("The average clustering coefficient of H is")
nx.average_clustering(H)

## Average sorthest path length
#### Warning! Calculating the shortest paths is intensive! 

The graph is small world.

In [ ]:
nx.average_shortest_path_length(H)

In [ ]:
math.log(len(H))

### Compare the results with a random ER network

We generate a random Erdos-Renyi graph with same average connectivity of H, i.e. same number of nodes and edges.

In [ ]:
nnodes=18000
plink=0.00122

ER=nx.fast_gnp_random_graph(nnodes, plink)

In [ ]:
nx.is_connected(ER)

In [ ]:
print("The ER graph has", len(ER), "nodes")
print("and",len(ER.edges()),"edges")

In [ ]:
print("The average clustering coefficient of ER is")
nx.average_clustering(ER)

In [ ]:
print(sum(list(nx.triangles(ER).values()))/3)

The ER graph is also small world!

In [ ]:
nx.average_shortest_path_length(ER)

### Compare the results with a random AB network

In [ ]:
AB=nx.barabasi_albert_graph(18000,11)

In [ ]:
print("The AB graph has", len(AB), "nodes")
print("and",len(AB.edges()),"edges")

In [ ]:
from collections import Counter 
degrees=dict(AB.degree()).values()
c=Counter(degrees)

In [ ]:
import powerlaw as pwl

In [ ]:
plt.figure(figsize=(10,7))
x=[]
y=[]
for i in sorted(c):   
    x.append(i)
    y.append(float(c[i])/len(AB))

    
plt.plot(np.array(x),np.array(y))
pwl.plot_pdf(list(degrees))

plt.xlabel('$k$', fontsize=18)
plt.ylabel('$P(k)$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.yscale('log')
plt.xscale('log')
plt.axis([10,1000,0.0000001,1])
plt.show()

In [ ]:
fit_function = pwl.Fit(list(degrees), xmin=11)

In [ ]:
fit_function.power_law.alpha

In [ ]:
fit_function.power_law.sigma

In [ ]:
fit_function.power_law.xmin

In [ ]:
print("The average clustering coefficient of AB is")
nx.average_clustering(AB)

In [ ]:
print("The number of triangles is ", sum(list(nx.triangles(AB).values()))/3)

The AB network is also small-world.

In [ ]:
nx.average_shortest_path_length(AB)

### Compare the results with a random WS network

In [ ]:
WS=nx.connected_watts_strogatz_graph(18000,23,0.2,50)

In [ ]:
print("The WS graph has", len(WS), "nodes")
print("and",len(WS.edges()),"edges")

In [ ]:
nx.is_connected(WS)

In [ ]:
ws_degrees=(dict(WS.degree()).values())

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(ws_degrees, bins=10)
plt.xlabel('$k$', fontsize=18)
plt.ylabel('$Freq.$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

In [ ]:
print("The average clustering coefficient of WS is")
nx.average_clustering(WS)

The Watt-Strogatz network is still small-world but with high clustering.

In [ ]:
nx.average_shortest_path_length(WS)

## Closeness Centrality

In connected graphs there is a natural distance metric between all pairs of nodes, defined by the length of their shortest paths. 
The '''farness''' of a node ''x'' is defined as the sum of its distances from all other nodes, and its closeness was defined by Bavelas as the reciprocal of the farness that is:


<center>
$C(x)= \frac{1}{\sum_y d(y,x)}.$
</center>


Thus, the more central a node is the lower its total distance from all other nodes. Note that taking distances ''from'' or ''to'' all other nodes is irrelevant in undirected graphs, whereas in directed graphs distances ''to'' a node are considered a more meaningful measure of centrality, as in general (e.g., in, the web) a node has little control over its incoming links.


#### Be careful! Computing all the distances between pair of nodes can be intensive.

In [ ]:
close_centr=nx.closeness_centrality(H)

In [ ]:
print(close_centr)

# Degree assortativity of a network

Assortativity can be measured in different ways. A simple approach is measuring the average nearest neighbor degree to assess the level of degree-assortativity.

In [ ]:
from collections import defaultdict

In [ ]:
x=[]
y=[]

avg_knn=defaultdict(list)

for n in G.nodes():
    k=G.degree(n)
    
    #nn=len(G.neighbors(n))
    total=0
    for j in G.neighbors(n):
        total+=G.degree(j)
    
    avg_knn[k].append(float(total)/k)
    
    x.append(k)
    y.append(float(total)/k)

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(x,y)

plt.xlabel('$k_i$', fontsize=18)
plt.ylabel('$k_{nn}$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.yscale('log')
plt.xscale('log')
plt.axis([0.1,1000,1,1000])
plt.show()

In [ ]:
z=[]
for k in sorted(avg_knn.keys()):
    knn=np.array(avg_knn[k])
    z.append(np.average(knn))

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(x,y)
plt.plot(sorted(avg_knn.keys()), z,'-r')

plt.xlabel('$k_i$', fontsize=18)
plt.ylabel('$k_{nn}$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.yscale('log')
plt.xscale('log')
plt.axis([0.1,1000,1,1000])
plt.show()

In [ ]:
r=nx.degree_assortativity_coefficient(G)

In [ ]:
print(r)

The degree assortativity coefficient of a ER graph is zero. The ER graph has no correlations.

In [ ]:
ERr=nx.degree_assortativity_coefficient(ER)

In [ ]:
ERr

NetworkX offers a number of functions to compute the same quantity.

In [ ]:
knn_avg2=nx.average_degree_connectivity(G)

In [ ]:
print(knn_avg2)

In [ ]:
knn_avg3=nx.k_nearest_neighbors(G)

In [ ]:
print(knn_avg3)

In [ ]:
r2=nx.degree_pearson_correlation_coefficient(G)
print(r2)

---
# Simulating random and targeted attacks to a network

Resilience is the ability to provide and maintain an acceptable level of service in the face of faults and challenges to normal operation. Threats and challenges for services can range from simple misconfiguration over large scale natural disasters to targeted attacks.

We define a function that performs a random or targeted attack to a network according to a given strategy (random, degree based, betweenness based, etc. )

In [ ]:
def net_attack(graph, ranked_nodes):
    
    fraction_removed=[]#here we store the tuples: (%removed nodes, size of gcc)
    
    graph1=graph.copy()
    nnodes=len(ranked_nodes)
    n=0    
    
    gcc=list(nx.connected_components(graph1))[0]
    
    gcc_size=float(len(gcc))/nnodes
    
    fraction_removed.append( (float(n)/nnodes, gcc_size) )
    
    while gcc_size>0.01:
        
        #we start from the end of the list!
        graph1.remove_node(ranked_nodes.pop())

        gcc=list(nx.connected_components(graph1))[0]
        gcc_size=float(len(gcc))/nnodes
        n+=1
        fraction_removed.append( (float(n)/nnodes, gcc_size) )
    
    return fraction_removed

# Robustness of the US airport network
## Random attack

In [ ]:
filepath_air='./../network_data/USairport_2010.txt'

In [ ]:
G=nx.Graph()
fh=open(filepath_air,'r')
for line in fh.readlines():
    s=line.strip().split()
    G.add_edge(int(s[0]),int(s[1]))
fh.close()    

In [ ]:
airport_nodes=list(G.nodes())

In [ ]:
resilience_random=net_attack(G, airport_nodes)

## Betweenness based attack

In [ ]:
from operator import itemgetter

In [ ]:
airport_nodes_betw=[]

betw=nx.betweenness_centrality(G)
for i in sorted(betw.items(), key=itemgetter(1)):
    airport_nodes_betw.append(i[0])


resilience_betw=net_attack(G, airport_nodes_betw)

## Degree based attack

In [ ]:
airport_nodes_degree=[]

deg=dict(G.degree())
for i in sorted(deg.items(), key=itemgetter(1)):
    airport_nodes_degree.append(i[0])


resilience_deg=net_attack(G, list(airport_nodes_degree))

Let's compare the results.

In [ ]:
x=[k[0] for k in resilience_random]
y=[k[1] for k in resilience_random]

x1=[k[0] for k in resilience_deg]
y1=[k[1] for k in resilience_deg]

x2=[k[0] for k in resilience_betw]
y2=[k[1] for k in resilience_betw]

plt.figure(figsize=(10,7))

plt.plot(x,y, label='random attack')
plt.plot(x1,y1, label='degree based')
plt.plot(x2,y2, label='betw based')

plt.xlabel('$f_{c}$', fontsize=18)
plt.ylabel('$LCC$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.legend(loc='upper right')

# Robustness of the Erdos-Renyi random network

In [ ]:
ER=nx.fast_gnp_random_graph(2000,0.012)

In [ ]:
ER_nodes=list(ER.nodes())

ER_nodes_deg=[i for i,d in sorted(dict(ER.degree()).items(), key=itemgetter(1))]

ER_betw=nx.betweenness_centrality(ER)
ER_nodes_betw=[i for i,b in sorted(dict(ER.ER_betw()).items(), key=itemgetter(1))

In [ ]:
resilience_random=net_attack(ER, ER_nodes)
resilience_deg=net_attack(ER, ER_nodes_deg)
resilience_betw=net_attack(ER, ER_nodes_betw)

In [ ]:
x=[k[0] for k in resilience_random]
y=[k[1] for k in resilience_random]

x1=[k[0] for k in resilience_deg]
y1=[k[1] for k in resilience_deg]

x2=[k[0] for k in resilience_betw]
y2=[k[1] for k in resilience_betw]

plt.figure(figsize=(10,7))

plt.plot(x,y, label='random attack')
plt.plot(x1,y1, label='degree based')
plt.plot(x2,y2, label='betw based')

plt.xlabel('$f_{c}$', fontsize=18)
plt.ylabel('$LCC$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.legend(loc='upper right')